In [260]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from langdetect import detect
import re

In [121]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lorih\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lorih\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lorih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [163]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [54]:
df = pd.read_csv('fifa_world_cup_2022_tweets.csv')

In [55]:
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
0,0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,neutral
1,1,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,positive
2,2,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,positive
3,3,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,positive
4,4,2022-11-20 23:58:28+00:00,0,Twitter for Android,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k",negative


In [4]:
df.shape

(22524, 6)

In [105]:
def preprocess(text):
    new_text = []
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9\s,.\'!?@#-]', '', text)
    text_list = [t for t in text.split(" ") if len(t) != 0]
    for t in text_list:   
        t = '' if t.startswith('@') else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [107]:
for i in range(10):
    text = df.iloc[i, 4]
    print(' '.join(text.split()))
    text_n = preprocess(text)
    print(text_n)
    print('----')

What are we drinking today @TucanTribe @MadBears_ @lkinc_algo @al_goanna #WorldCup2022 https://t.co/Oga3TzvG5h
What are we drinking today     #WorldCup2022 
----
Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action! This is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8
Amazing  #WorldCup2022 launch video. Shows how much the face of Canada and our mens national team have changed since our last World Cup entry in 1986. Cant wait to see these boys in action! This is Canada FIFA World Cup Opening Video 
----
Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU
Worth reading while watching #WorldCup2022 
----
Golden Maknae shinning bright https://t.co/4AyZbzGTX4 #JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK @BTS_twt
Golden Maknae shinning bright 

In [108]:
df['Tweet_cleaned'] = df['Tweet'].apply(preprocess)

In [109]:
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment,Tweet_cleaned
0,0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,neutral,What are we drinking today #WorldCup2022
1,1,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,positive,Amazing #WorldCup2022 launch video. Shows how much the face of Canada and our mens national team have changed since our last World Cup entry in 1986. Cant wait to see these boys in action! This is Canada FIFA World Cup Opening Video
2,2,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,positive,Worth reading while watching #WorldCup2022
3,3,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,positive,Golden Maknae shinning bright #JeonJungkook #Jungkook # # #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS # #WorldCup2022 #FIFAKOOK
4,4,2022-11-20 23:58:28+00:00,0,Twitter for Android,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k",negative,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony??"


In [110]:
lemmatizer = WordNetLemmatizer()

In [117]:
def preprocess_text(text):
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalpha() 
              and word.lower() not in stopwords.words('english')]
    return " ".join(tokens)

In [124]:
df['token'] = df['Tweet_cleaned'].apply(preprocess_text)

In [125]:
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment,Tweet_cleaned,token
0,0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadBears_ \n@lkinc_algo \n@al_goanna \n\n#WorldCup2022 https://t.co/Oga3TzvG5h,neutral,What are we drinking today #WorldCup2022,drinking today
1,1,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch video. Shows how much the face of Canada and our men’s national team have changed since our last World Cup entry in 1986. Can’t wait to see these boys in action!\n\nThis is Canada: FIFA World Cup Opening Video https://t.co/7g73vvwtg8,positive,Amazing #WorldCup2022 launch video. Shows how much the face of Canada and our mens national team have changed since our last World Cup entry in 1986. Cant wait to see these boys in action! This is Canada FIFA World Cup Opening Video,amazing launch video show much face canada men national team changed since last world cup entry cant wait see boy action canada fifa world cup opening video
2,2,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 https://t.co/1SQrNa2dYU,positive,Worth reading while watching #WorldCup2022,worth reading watching
3,3,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/4AyZbzGTX4\n#JeonJungkook #Jungkook #전정국 #정국 #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS #방탄소년단 #WorldCup2022 #FIFAKOOK \n@BTS_twt,positive,Golden Maknae shinning bright #JeonJungkook #Jungkook # # #JK #GoldenMaknae #bunny #Kookie #Jungshook #BTS # #WorldCup2022 #FIFAKOOK,golden maknae shinning bright jeonjungkook jungkook jk goldenmaknae bunny kookie jungshook bts fifakook
4,4,2022-11-20 23:58:28+00:00,0,Twitter for Android,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony?? https://t.co/f72P03ZN2k",negative,"If the BBC cares so much about human rights, homosexual rights, and women rights then why not say these before the opening ceremony?? Why are they saying these during the opening ceremony?? Why did the BBC censor the #WorldCup2022 opening ceremony??",bbc care much human right homosexual right woman right say opening ceremony saying opening ceremony bbc censor opening ceremony


In [269]:
df.shape

(22524, 8)

In [276]:
mask = np.random.rand(len(df)) < 0.99
df_train_val = df[mask]
df_test = df[~mask]

In [277]:
# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=20000)  
X = vectorizer.fit_transform(df_train_val['token'])
y = df_train_val['Sentiment']

In [278]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [279]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [280]:
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

    negative       0.75      0.62      0.68      1133
     neutral       0.67      0.64      0.65      1691
    positive       0.69      0.80      0.74      1634

    accuracy                           0.69      4458
   macro avg       0.70      0.69      0.69      4458
weighted avg       0.70      0.69      0.69      4458



In [281]:
df_test.shape

(237, 8)

In [283]:
X_test = vectorizer.transform(df_test['token'])
y_test = df_test['Sentiment']
y_test_pred = model.predict(X_test)

In [284]:
X_test_df = df_test[['Tweet_cleaned', 'token']].reset_index(drop = True)
y_test_df = df_test[['Sentiment']].reset_index(drop = True)
y_test_df.columns = ['True_Label']
y_test_pred_df = pd.DataFrame(y_test_pred).reset_index(drop = True)
y_test_pred_df.columns = ['Predicted_Label']

In [285]:
comparison_df = pd.concat([X_test_df, y_test_df, y_test_pred_df], axis=1)

In [286]:
comparison_df.isna().sum()

Tweet_cleaned      0
token              0
True_Label         0
Predicted_Label    0
dtype: int64

In [287]:
comparison_df['diff'] = comparison_df.apply(
    lambda row: 'diff' if row['True_Label'] != row['Predicted_Label'] else 'same', axis = 1)

In [288]:
comparison_df[comparison_df['diff'] == 'diff'].shape

(77, 5)

In [290]:
comparison_df[comparison_df['diff'] == 'diff'].head(20)

,Tweet_cleaned,token,True_Label,Predicted_Label,diff
2,I'm in on the Gaffer's #WorldCup2022 Sweepstakes where supporting an underdog is a winning strategy! From #GaffersUnderdogs,gaffer sweepstakes supporting underdog winning strategy gaffersunderdogs,positive,neutral,diff
4,It already started #WorldCup2022 #Catar2022 #qtar2022,already started,neutral,negative,diff
5,"Thats first goal , was that really offside? And he went on and scored 2 more #WorldCup2022 #Qatar2022",thats first goal really offside went scored,positive,negative,diff
7,Here is my world cup fantasy selection. missed the deadline and forgot to on 12th man booster. all the best to all the lovely people. #WorldCupFantasy #Worldcup #worldcup2022 #QatarWorldCup2022 #FIFAWorldCup,world cup fantasy selection missed deadline forgot man booster best lovely people worldcupfantasy worldcup fifaworldcup,neutral,positive,diff
8,"My professors cant expect me to go to class during the World Cup, can they? #WorldCup2022",professor cant expect go class world cup,negative,positive,diff
12,On me head son #WorldCup2022,head son,neutral,positive,diff
16,Game one in the books #WorldCup2022,game one book,neutral,positive,diff
17,"The #QatarWorldCup2022 openers in the books, and if youre after more info about every team and group, all 8 of our group previews are up on our feed! And we have a special BONUS EPISODE with predictions, trivia, and more. #EcuadorvsQatar #WorldCup2022",opener book youre info every team group group preview feed special bonus episode prediction trivia ecuadorvsqatar,positive,neutral,diff
19,Today was the first time I ever watch Qatar play. And I cancelled my bet when I saw that first touch. #Worlds2022 #WorldCup2022 #QatarWorldCup2022 #QatarvsEcuador,today first time ever watch qatar play cancelled bet saw first touch qatarvsecuador,negative,neutral,diff
30,"I am just wondering,if a call for prayer starts right now,will Qatar players put the game on hold to appease allah? Religion ID stupidity.#WorldCup2022",wondering call prayer start right qatar player put game hold appease allah religion id,negative,positive,diff


In [291]:
comparison_df[comparison_df['diff'] == 'same'].shape

(160, 5)

In [292]:
comparison_df[comparison_df['diff'] == 'same'].head(20)

,Tweet_cleaned,token,True_Label,Predicted_Label,diff
0,Exciting things are happening at #Sporten headquarters. Dont miss the opportunity and see you at the #WorldCup2022 !!!! #FIFAWorldCupQatar2022 #QatarWorldCup2022 #Airdrop BALL,exciting thing happening sporten headquarters dont miss opportunity see airdrop ball,positive,positive,same
1,Fans across the Middle East celebrate historic World Cup moment #Gaza #Qatar2022 #WorldCup2022,fan across middle east celebrate historic world cup moment gaza,positive,positive,same
3,"FAKE Football fan I hate BlackBrown people, but I hate Arabs too. What can I do at the #WorldCup2022 , without being identified as a #Racist. I know,... I will put out a STUPID #BotcottQatar hastag and pretend to be an activist",fake football fan hate blackbrown people hate arab without identified racist know put stupid botcottqatar hastag pretend activist,negative,negative,same
6,So it's #Pakistani footballs worth it. But recitation of #HolyQuran in 92 years is something worth more then once in life time history. #WorldCup2022 #FIFAWorldCup #FIFAWorldCupQatar2022 #FIFA,pakistani football worth recitation holyquran year something worth life time history fifaworldcup fifa,positive,positive,same
9,"Morocco vs Croatia Live Stream Live On Morocco vs Croatia Wednesday, November 23, 2022 1000 AM GMT Al Bayt Stadium, Al Khor, Qatar FIFA World Cup 2022 #Morocco #FIFAWorldCup #Qatar #QatarWorldCup2022 #Qatar2022 #WorldCup2022 #Croatia #FIFA",morocco v croatia live stream live morocco v croatia wednesday november gmt al bayt stadium al khor qatar fifa world cup morocco fifaworldcup qatar croatia fifa,neutral,neutral,same
10,Can someone tell me when Whales is playing Brazil in the World Cup? This will be the best match of the entire games. #WorldCup2022 #WorldCup #Brazil #soccer #FIFAWorldCup,someone tell whale playing brazil world cup best match entire game worldcup brazil soccer fifaworldcup,positive,positive,same
11,Victory is a State of Mind. #LionelMessi and for The big day came. World Cup started! Which team do you support in this cup? #WorldCup2022 #Qatar2022 #realtimemarketing,victory state mind lionelmessi big day came world cup started team support cup realtimemarketing,positive,positive,same
13,AF Poster LAST HURRAH This World Cup may be the last for some of the icons of our generation. #worldcup #worldcup2022 #qatarworldcup #afposter,af poster last hurrah world cup may last icon generation worldcup qatarworldcup afposter,positive,positive,same
14,are you at this #WorldCup2022 to win it or to make a stance for racism and gays ???? Its a complete shambles!! We will fall early because of this shit pc approach!! #FIFAWorldCup,win make stance racism gay complete shamble fall early shit pc approach fifaworldcup,negative,negative,same
15,This by far is probably the best Twitter exchange I've seen as far as the #WorldCup2022 goes. #Qatar2022 #Qatar,far probably best twitter exchange seen far go qatar,positive,positive,same
